In [1]:
# Install dependencies
!pip install fastapi uvicorn pyngrok llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.2-cp310-cp310-linux_x86_64.whl size=3410012 sha256=3c41ea2c286808bf1282cef2684cc7146ce3a963a895f62e907f646287130f6b
  Stored in directory: /root/.cache/pip/wheels/49/1c/f0/6c1ed6032d5827dea28df8a8df860d90ca1bcd7b3fa455f5e1
Successfully built llama-cpp-python


In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary modules
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from llama_cpp import Llama
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from threading import Thread

In [4]:
# Allow nested event loops in Colab
nest_asyncio.apply()

# Initialize FastAPI
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Izinkan permintaan dari origins yang ditentukan
    allow_credentials=True,
    allow_methods=["*"],  # Izinkan semua metode HTTP (GET, POST, PUT, DELETE, dll.)
    allow_headers=["*"],  # Izinkan semua header
)

In [5]:
# Path model yang telah disalin ke Google Drive
model_path = "/content/drive/MyDrive/llama_model/unsloth.Q4_K_M.gguf"

# Memuat model dari Google Drive
llm = Llama(model_path)

llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /content/drive/MyDrive/llama_model/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8b Bnb 4bit
llama_model_loader: - kv   3:                       general.organization str              = Unsloth
llama_model_loader: - kv   4:                           general.finetune str              = bnb-4bit
llama_model_loader: - kv   5:                           general.basename str              = meta-llama-3.1
llama_model_loader: - kv   6:                         general.size_label str              = 8B
llama_model_loader: 

In [6]:
# Define request and response models
class ChatRequest(BaseModel):
    instruction: str
    input_data: str = ""

class ChatResponse(BaseModel):
    response: str

# Alpaca-style prompt template
alpaca_prompt = """Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan masukan yang memberikan konteks lebih lanjut. Tulis tanggapan yang melengkapi permintaan dengan tepat.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [7]:
# Root endpoint
@app.post("/chat", response_model=ChatResponse)
async def chat_completion(request: ChatRequest):
    # Format the instruction and input for the model
    prompt = alpaca_prompt.format(request.instruction, request.input_data, "")

    # Generate the chat completion
    result = llm.create_chat_completion(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    # Extract and return the result
    response_text = result['choices'][0]['message']['content']

    return ChatResponse(response=response_text)

@app.get("/")
async def root():
    return {"message": "LLM Model is successfully running"}

In [8]:
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
# Run the server in a separate thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the FastAPI server
thread = Thread(target=run)
thread.start()

Public URL: NgrokTunnel: "https://c404-34-48-125-98.ngrok-free.app" -> "http://localhost:8000"
